#Method 1







In [0]:
#!pip install twitter
!pip install python-twitter

    100% |████████████████████████████████| 71kB 2.7MB/s 


In [0]:
import  twitter

### Acessing Twitter API

In [0]:
# initialize api instance
twitter_api = twitter.Api(consumer_key='XXXXXXXXXXXX',
                        consumer_secret='XXXXXXXXXXXXXXXXXXXXXXXXXXXX',
                        access_token_key='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
                        access_token_secret='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# test authentication
print(twitter_api.VerifyCredentials())

{"created_at": "Wed Apr 03 08:55:05 +0000 2019", "default_profile": true, "followers_count": 4, "friends_count": 13, "id": 1113364285957107713, "id_str": "1113364285957107713", "lang": "en", "name": "WRITAM MALLIK", "profile_background_color": "F5F8FA", "profile_image_url": "http://pbs.twimg.com/profile_images/1119465955044859905/aEfR3qsi_normal.png", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1119465955044859905/aEfR3qsi_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "MallikWritam"}


###Building Test Dataset

In [0]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [0]:
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword:Modi
Fetched 100 tweets for the term Modi
[{'text': 'For my understanding. RT if you are a core BJP/Modi supporter and are DELIGHTED with the @BJP4India fielding Sadhvi… https://t.co/M4ZCoUmf17', 'label': None}, {'text': 'PM Modi fiercely defends giving ticket of Sadhvi Pragya Thakur in his interview to Times Now. Not only he says she… https://t.co/G3grHOb2M2', 'label': None}, {'text': 'Suspended IAS officer Mohsin who tried to inspect PM’s copter is a perennial BJP/ Modi hater and was slammed by Cou… https://t.co/ouSwAu7Hft', 'label': None}, {'text': 'RT @bhak_sala: Monday:\nSupreme Court serves Rahul Gandhi notice, says, ‘We never said anything about Modi’. The Supreme court gave Rahul Ga…', 'label': None}]


###Building Training Dataset

In [0]:
def buidTrainingSet(corpusFile, tweetDataFile):
    import csv
    import time
    
    corpus = []
    
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
            
    rate_limit = 180
    sleep_time = 900/180
    
    trainingDataSet = []
    
    for tweet in corpus:
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time) 
        except: 
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'w') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [0]:
corpusFile = "corpus.csv"
tweetDataFile = "tweetDataFile.csv"

trainingData = buidTrainingSet(corpusFile, tweetDataFile)

Tweet fetchedHilarious @youtube video - guy does a duet with @apple 's Siri. Pretty much sums up the love affair! http://t.co/8ExbnQjY
Tweet fetched@RIM you made it too easy for me to switch to @Apple iPhone. See ya!
Tweet fetchedThe 16 strangest things Siri has said so far. I am SOOO glad that @Apple gave Siri a sense of humor! http://t.co/TWAeUDBp via @HappyPlace
Tweet fetchedGreat up close & personal event @Apple tonight in Regent St store!
Tweet fetchedFrom which companies do you experience the best customer service aside from @zappos and @apple?
Tweet fetchedLmao I think @apple is onto something magical! I am DYING!!! haha. Siri suggested where to find whores and where to hide a body lolol
Tweet fetchedRT @PhillipRowntree: Just registered as an @apple developer... Here's hoping I can actually do it... Any help, greatly appreciated!
Tweet fetchedWow. Great deals on refurbed #iPad (first gen) models. RT: Apple offers great deals on refurbished 1st-gen iPads http://t.co/ukWOKBGd @App

### Importing neccessary Dependencies

In [0]:
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('punkt')
from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


###Preprocessing

In [0]:
class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self.preprocessTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def preprocessTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

In [0]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

###Feature Extraction

In [0]:
#Building the vocabulary

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [0]:
#Matching tweets against our vocabulary

def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [0]:
#Building our feature vector

word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

###Classification

In [0]:
#Training the classifier

NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

In [0]:
#Testing The Model

NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

Overall Negative Sentiment
Negative Sentiment Percentage = 1.0%


#Method 2

In [0]:
#Import the necessary methods from tweepy library as well as json and time
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import json
import time
import pandas as pd

In [0]:
#We authenticate ourselves as having a twitter app
#Variables that contains the user credentials to access Twitter API 
access_token = "1113364285957107713-ifuwPJ2GRnizO1iRJ5UklnjIAPSMiK"
access_secret = "kDK8xD0UDES0UHKC2lEhVsR8GucNTYQ8h5YUzaSA8xGVI"
consumer_key = "YME7NUbcWc8hXtiYAqkSqrsLP"
consumer_secret = "d3qaV99wr87RDdUrP6Ur4oN9SNCh3PLCA9VLsxNMA46DJyK1jD"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

### **Getting Data**

In [0]:
#We begin searching our query
#Put your search term
searchquery = "angry"

users =tweepy.Cursor(api.search,q=searchquery).items()
count = 0
start = 0
errorCount=0

#We will be storing our data in file called: happy.json
#file = open('test.json', 'wb') 

#here we tell the program how fast to search 
waitquery = 100      #this is the number of searches it will do before resting
waittime = 2.0          # this is the length of time we tell our program to rest
total_number = 15000     #this is the total number of queries we want
justincase = 1         #this is the number of minutes to wait just in case twitter throttles us

In [0]:
text = [0] * total_number
secondcount = 0
idvalues = [1] * total_number
 #1 is happy; 2 is sad; 3 is angry; 4 is fearful
#Below is where the magic happens and the queries are being made according to our desires above
while secondcount < total_number:
    try:
        user = next(users)
        count += 1
        
        #We say that after every 100 searches wait 5 seconds
        if (count%waitquery == 0):
            time.sleep(waittime)
            #break

    except tweepy.TweepError:
        #catches TweepError when rate limiting occurs, sleeps, then restarts.
        #nominally 15 minnutes, make a bit longer to avoid attention.
        print("sleeping....")
        time.sleep(60*justincase)
        user = next(users)
        
        
    except StopIteration:
        break
    try:
        #print "Writing to JSON tweet number:"+str(count)
        text_value = user._json['text']
        language = user._json['lang']
        #print(text_value)
        print(language)
        
        if "RT" not in text_value:
            if language == "en":
                text[secondcount] = text_value
                secondcount = secondcount + 1
                print("current saved is:")
                print(secondcount)

    except UnicodeEncodeError:
        errorCount += 1
        print("UnicodeEncodeError,errorCount ="+str(errorCount))

print("Creating dataframe:")

d = {"text": text, "id": idvalues}
df = pd.DataFrame(data = d)

df.to_csv('upset.csv', header=True, index=False, encoding='utf-8')

print("completed")

en
en
en
current saved is:
1
en
en
en
hi
en
current saved is:
2
en
current saved is:
3
en
en
current saved is:
4
en
current saved is:
5
en
current saved is:
6
en
en
en
en
en
pt
en
current saved is:
7
en
current saved is:
8
en
current saved is:
9
en
current saved is:
10
und
en
und
en
current saved is:
11
en
current saved is:
12
en
en
en
current saved is:
13
en
en
current saved is:
14
en
current saved is:
15
en
current saved is:
16
en
en
current saved is:
17
en
current saved is:
18
en
en
en
en
current saved is:
19
en
current saved is:
20
ko
en
en
en
current saved is:
21
en
current saved is:
22
en
current saved is:
23
en
ja
en
en
current saved is:
24
und
en
current saved is:
25
en
en
current saved is:
26
ko
en
en
en
current saved is:
27
en
current saved is:
28
en
en
en
en
en
current saved is:
29
en
en
current saved is:
30
en
ru
en
en
current saved is:
31
en
en
current saved is:
32
en
en
en
current saved is:
33
en
current saved is:
34
en
current saved is:
35
en
en
en
current saved is:
36
e

Rate limit reached. Sleeping for: 809


en
en
current saved is:
501
en
en
current saved is:
502
en
current saved is:
503
en
current saved is:
504
en
en
en
en
en
en
current saved is:
505
en
en
en
en
en
current saved is:
506
en
current saved is:
507
en
en
en
en
current saved is:
508
en
current saved is:
509
en
current saved is:
510
tl
en
en
current saved is:
511
ko
en
current saved is:
512
en
en
en
en
current saved is:
513
en
current saved is:
514
en
en
en
en
en
current saved is:
515
en
en
en
en
current saved is:
516
en
current saved is:
517
en
en
current saved is:
518
en
en
current saved is:
519
en
en
current saved is:
520
en
en
current saved is:
521
en
en
current saved is:
522
en
en
en
current saved is:
523
en
current saved is:
524
en
current saved is:
525
en
current saved is:
526
en
en
current saved is:
527
en
en
en
current saved is:
528
en
current saved is:
529
en
current saved is:
530
en
en
en
current saved is:
531
en
en
current saved is:
532
ko
en
current saved is:
533
en
current saved is:
534
en
current saved is:
535
en

Rate limit reached. Sleeping for: 833


en
en
en
en
ru
en
en
en
current saved is:
1388
en
en
current saved is:
1389
en
current saved is:
1390
en
current saved is:
1391
en
current saved is:
1392
en
current saved is:
1393
en
en
en
en
current saved is:
1394
en
en
en
current saved is:
1395
en
current saved is:
1396
en
current saved is:
1397
en
current saved is:
1398
en
current saved is:
1399
en
en
current saved is:
1400
en
en
en
en
en
current saved is:
1401
en
en
current saved is:
1402
en
en
current saved is:
1403
en
en
en
current saved is:
1404
en
current saved is:
1405
en
en
current saved is:
1406
en
en
en
en
en
en
en
en
current saved is:
1407
en
current saved is:
1408
en
current saved is:
1409
en
current saved is:
1410
en
current saved is:
1411
en
en
en
en
current saved is:
1412
en
current saved is:
1413
en
en
en
current saved is:
1414
en
current saved is:
1415
en
en
current saved is:
1416
en
en
en
en
en
current saved is:
1417
en
en
en
en
en
en
en
en
en
current saved is:
1418
en
en
en
current saved is:
1419
en
current saved i

...

en
current saved is:
12933
en
ko
en
current saved is:
12934
en
en
en
current saved is:
12935
en
en
current saved is:
12936
en
en
en
current saved is:
12937
en
en
en
en
en
current saved is:
12938
en
en
en
current saved is:
12939
en
en
current saved is:
12940
en
en
current saved is:
12941
en
en
en
current saved is:
12942
en
en
current saved is:
12943
en
current saved is:
12944
en
current saved is:
12945
en
en
current saved is:
12946
en
en
current saved is:
12947
en
en
en
en
en
en
current saved is:
12948
en
current saved is:
12949
en
en
current saved is:
12950
en
current saved is:
12951
en
current saved is:
12952
en
en
en
current saved is:
12953
en
en
en
en
en
en
current saved is:
12954
en
en
current saved is:
12955
ja
hi
en
en
en
current saved is:
12956
en
en
en
es
en
current saved is:
12957
en
en
en
current saved is:
12958
en
current saved is:
12959
en
en
current saved is:
12960
en
current saved is:
12961
en
en
en
en
en
current saved is:
12962
en
current saved is:
12963
en
current saved

Rate limit reached. Sleeping for: 829


en
en
en
en
current saved is:
13718
en
en
current saved is:
13719
en
en
en
en
en
current saved is:
13720
en
en
en
en
current saved is:
13721
en
en
en
current saved is:
13722
en
en
en
current saved is:
13723
en
current saved is:
13724
en
current saved is:
13725
en
current saved is:
13726
en
en
en
en
current saved is:
13727
en
en
en
en
current saved is:
13728
en
en
current saved is:
13729
en
en
en
current saved is:
13730
en
current saved is:
13731
en
en
en
en
en
en
en
current saved is:
13732
en
en
en
en
current saved is:
13733
en
en
current saved is:
13734
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
current saved is:
13735
en
en
en
current saved is:
13736
en
current saved is:
13737
en
current saved is:
13738
en
current saved is:
13739
en
current saved is:
13740
en
en
en
en
en
en
current saved is:
13741
en
en
en
en
en
current saved is:
13742
en
en
en
current saved is:
13743
en
current saved is:
13744
en
current saved is:
13745
en
current saved is:
13746
en
en
en
en
current saved is

Rate limit reached. Sleeping for: 828


en
en
current saved is:
14553
en
current saved is:
14554
en
en
current saved is:
14555
en
current saved is:
14556
en
en
en
en
en
en
current saved is:
14557
en
en
en
en
en
en
current saved is:
14558
en
en
current saved is:
14559
en
en
current saved is:
14560
en
current saved is:
14561
en
en
current saved is:
14562
en
en
current saved is:
14563
en
en
en
en
current saved is:
14564
en
en
en
en
en
current saved is:
14565
en
current saved is:
14566
en
current saved is:
14567
en
en
current saved is:
14568
en
en
en
en
en
en
current saved is:
14569
en
current saved is:
14570
en
current saved is:
14571
en
current saved is:
14572
en
en
en
en
en
current saved is:
14573
en
current saved is:
14574
en
en
en
en
en
current saved is:
14575
pl
en
en
en
en
en
pt
en
en
en
en
en
en
current saved is:
14576
en
en
current saved is:
14577
en
en
en
en
en
en
en
current saved is:
14578
en
en
current saved is:
14579
in
en
en
en
en
en
current saved is:
14580
en
current saved is:
14581
en
en
en
current saved is:
1458

### **Cleaning the data and getting the words that appear**

In [0]:
import pandas as pd
import string

In [0]:
#import my csv file
df = pd.read_csv('unsmile.csv')

In [0]:
#Remove any rows with a "nan" in them
df = df.dropna(axis=0, how = 'any')

In [0]:
#Make it so that any non readable text gets converted into nothing
def removetext(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

In [0]:
#Here I am doing the actual removing
df['text'] = df['text'].apply(removetext)

#Make all my texts lower case
df['text'] = df['text'].apply(lambda x: x.lower())

In [0]:
#Get rid of all weird punctuation and extra lines
df['text'] = df['text'].apply(lambda x: x.replace('.',' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n',' '))
df['text'] = df['text'].apply(lambda x: x.replace('?',' '))
df['text'] = df['text'].apply(lambda x: x.replace('!',' '))
df['text'] = df['text'].apply(lambda x: x.replace('"',' '))
df['text'] = df['text'].apply(lambda x: x.replace(';',' '))
df['text'] = df['text'].apply(lambda x: x.replace('#',' '))
df['text'] = df['text'].apply(lambda x: x.replace('&amp',' '))
df['text'] = df['text'].apply(lambda x: x.replace(',',' '))

In [0]:
df1=df.copy()
df2=df.copy()

In [0]:
#split all the words in my text
df1['text']= df1['text'].str.split()

df1.to_csv('unsmile_split.csv')

In [0]:
#Here I get each unique keyword from my dataframe
array = df2['text'].str.split(' ', expand=True).stack().value_counts()
#print(array) to see what this looks like

In [0]:
#I make a dataframe of the words and the frequency with which the words appear 
d = {'word': array.index, 'frequency':array}
df2 = pd.DataFrame(data = d)

In [0]:
#I get rid of any words that are mentioned less than 10 times
df2['frequency'] = df2['frequency'][df2['frequency'] > 10]

In [0]:
#Remove any rows with a "nan" in them
df2 = df2.dropna(axis=0, how = 'any')

In [0]:
#Drop any obvious signs of these words being :(
df2 = df2.drop([':(','https://t',':((', ':(((', ':((((', ':(((((', ':', '(', ''])

In [0]:
#Convert my dataframe into a csv file
df2.to_csv('unsmile_words.csv', header=True, index=False, encoding='utf-8')

**Same Process for unsmile,fun,happy,sad**

### **Get the bag of words**

In [0]:
#import my csv files
happy = pd.read_csv('happy_words.csv')
sad = pd.read_csv('sad_words.csv')
unsmile = pd.read_csv('unsmile_words.csv')
fun = pd.read_csv('fun_words.csv')

In [0]:
wordbag = pd.concat([happy,sad,unsmile,fun]).drop_duplicates(subset = 'word').reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
wordbag

,frequency,word
0,5011.0,i
1,4987.0,to
2,4335.0,you
3,3824.0,birthday
4,3761.0,the
5,3657.0,and
6,3238.0,so
7,3162.0,a
8,2409.0,my
9,2135.0,for


In [0]:
wordbag.to_csv('wordbag.csv')

### **Create the frequency table**

In [0]:
#Read the files with the tweets
fun = pd.read_csv('fun_split.csv')
happy = pd.read_csv('happy_split.csv')
unsmile = pd.read_csv('unsmile_split.csv')
sad = pd.read_csv('sad_split.csv')

In [0]:
#Get the wordbag
wordbag = pd.read_csv('wordbag.csv')
wordbag = wordbag.drop_duplicates()

In [0]:
#Classify tweets that had the word fun or happy in them as 1 (positive)
#and the others as 0 (negative)
fun['type'] = 1
happy['type'] = 1
unsmile['type'] = 0
sad['type'] = 0

In [0]:
#Join all of the dataframes into one big one for easier manipulation of a test/train split
df = pd.concat([happy,sad,unsmile,fun]).reset_index(drop=True)

In [0]:
#Create a test and train set by using the sklearn function train_test_split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

In [0]:
#Seperate the train data into a positive and negative set
train_positive = train[train['type'] ==1]
train_negative = train[train['type'] ==0]

In [0]:
positive_instance = len(train_positive)
negative_instance = len(train_negative)
print(positive_instance)
print(negative_instance)

23883
24117


In [0]:
#Create your frequency table
frequency=[]
prev=''
for char in wordbag['word']:
    if char == prev:
        pass
    else:
        frequency.append(char)
        prev = char
print(frequency) 

['i', 'to', 'you', 'birthday', 'the', 'and', 'so', 'a', 'my', 'for', 'be', 'im', 'is', 'of', 'me', 'in', 'that', 'it', 'this', 'with', 'love', 'day', 'your', 'have', 'but', 'are', 'just', 'on', 'was', "i'm", 'all', 'hope', 'not', 'we', 'makes', 'at', 'good', 'make', 'as', 'one', 'if', 'like', 'when', 'see', 'am', 'about', 'u', 'what', 'can', 'they', 'its', 'he', 'how', 'get', 'do', 'life', 'much', 'know', 'really', 'great', 'very', 'been', 'will', 'up', 'now', 'best', 'today', 'more', 'from', 'people', 'always', 'out', 'time', 'her', 'bday', 'got', 'had', 'may', 'our', 'thank', '-', 'an', 'miss', 'dont', 'she', 'him', 'being', 'would', 'us', 'no', 'want', 'who', 'them', 'new', 'go', 'because', 'never', 'or', "it's", 'too', 'by', 'happy,', 'has', 'girl', 'back', 'cant', 'going', 'friday', 'feel', 'made', 'birthday,', 'man', 'think', 'his', 'wish', 'here', 'only', 'youre', 'everyone', 'star', 'year', 'heart', 'some', 'well', 'were', 'there', 'still', 'thanks', 'say', 'even', 'look', 'fri

In [0]:
word_bank = [0]*len(frequency)
positive = [0]*len(frequency)
negative = [0]*len(frequency)

In [0]:
#Go over all the words in the frequency table
for i in range(len(frequency)):
    
    #Get the word in the frequency table at a given row
    word = frequency['word'].iloc[i]
    word_bank[i] = word
             
    #Convert the word and attached single colons ont oboth sides of the word
    check = str("'") + word + str("'")
    
    #Count the number of instances that have the word at least once
    count = 0  
    
    #this iterates through each of the tweets in the positive train set
    for j in range(len(train_positive)):
        #This checks to see the number of time the said word appears in a given tweet
        appears = train_positive['text'].iloc[j].count(check)
        
        #If the word appears at least once, we count it as that tweet having it
        #We sum over all the tweets that the word appears at least once in 
        if appears > 0:
            count = count + 1
    positive[i] = count
    
    #Does the same thing but for negative numbers
    count = 0  
    for k in range(len(train_negative)):
        appears = train_negative['text'].iloc[k].count(check)
        if appears > 0:
            count = count + 1
    negative[i] = count
    print(i)